In [ ]:
import os
import json
import time
import asyncio
from datetime import datetime
from strands import Agent, tool
from strands.models.anthropic import AnthropicModel
import httpx
from typing import Dict, Any, List, Optional

# Configuration - set your API base URL here
BASE_URL = "http://localhost:8000"  # Change this to your deployed URL as needed

# ============= API-CALLING TOOLS =============

@tool
async def get_recent_financial_data(limit: int = 100) -> dict:
    """
    Get the most recent financial data rows for analysis.
    
    Args:
        limit (int): Number of recent rows to fetch (default: 100, max: 1000)
    
    Returns:
        dict: Financial data response with events
    """
    try:
        # Ensure limit doesn't exceed 1000
        limit = min(limit, 1000)
        
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"{BASE_URL}/api/agent-query",
                params={"limit": limit},
                timeout=30.0
            )
            response.raise_for_status()
            
            data = response.json()
            
            return {
                "status": "success",
                "timestamp": datetime.now().isoformat(),
                "data": data.get("events", data) if isinstance(data, dict) else data,
                "count": len(data.get("events", data) if isinstance(data, dict) else data),
                "message": f"Retrieved {len(data.get('events', data) if isinstance(data, dict) else data)} financial records"
            }
    
    except httpx.HTTPStatusError as e:
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "data": [],
            "count": 0,
            "message": f"HTTP {e.response.status_code} error: {str(e)}"
        }
    except Exception as e:
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "data": [],
            "count": 0,
            "message": f"Error retrieving financial data: {str(e)}"
        }

@tool
async def get_current_graphs() -> dict:
    """
    Get all currently displayed graphs from the dashboard.
    
    Returns:
        dict: Response containing list of current graphs with their configurations
    """
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"{BASE_URL}/api/graphs",
                timeout=30.0
            )
            response.raise_for_status()
            
            graphs = response.json()
            
            return {
                "status": "success",
                "timestamp": datetime.now().isoformat(),
                "graphs": graphs,
                "count": len(graphs),
                "message": f"Retrieved {len(graphs)} graphs from dashboard"
            }
    
    except httpx.HTTPStatusError as e:
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "graphs": [],
            "count": 0,
            "message": f"HTTP {e.response.status_code} error: {str(e)}"
        }
    except Exception as e:
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "graphs": [],
            "count": 0,
            "message": f"Error retrieving graphs: {str(e)}"
        }

@tool
async def remove_graph(graph_id: str) -> dict:
    """
    Remove a graph by its ID from the dashboard.
    
    Args:
        graph_id (str): ID of the graph to remove
    
    Returns:
        dict: Removal result with confirmation
    """
    try:
        async with httpx.AsyncClient() as client:
            response = await client.delete(
                f"{BASE_URL}/api/graphs/{graph_id}",
                timeout=30.0
            )
            response.raise_for_status()
            
            result = response.json()
            
            return {
                "status": "success",
                "timestamp": datetime.now().isoformat(),
                "removed_id": graph_id,
                "message": result.get("message", f"Successfully removed graph {graph_id}")
            }
    
    except httpx.HTTPStatusError as e:
        if e.response.status_code == 404:
            return {
                "status": "error",
                "timestamp": datetime.now().isoformat(),
                "removed_id": None,
                "message": f"Graph {graph_id} not found"
            }
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "removed_id": None,
            "message": f"HTTP {e.response.status_code} error: {str(e)}"
        }
    except Exception as e:
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "removed_id": None,
            "message": f"Error removing graph: {str(e)}"
        }

@tool
async def add_new_graph(type: str, title: str, sql_query: str, extra: Optional[Dict] = None) -> dict:
    """
    Add a new graph to the financial dashboard.
    
    Args:
        type (str): Type of graph (bar, line, pie, area, scatter, scatter3d)
        title (str): Title for the graph
        sql_query (str): SQL query to fetch data for the graph
        extra (Optional[Dict]): Additional parameters like axis labels
    
    Returns:
        dict: Addition result with new graph details
    """
    try:
        # Prepare the graph configuration
        graph_config = {
            "type": type,
            "title": title,
            "sql_query": sql_query
        }
        
        if extra:
            graph_config["extra"] = extra
        
        async with httpx.AsyncClient() as client:
            response = await client.post(
                f"{BASE_URL}/api/graphs",
                json=graph_config,
                timeout=30.0
            )
            response.raise_for_status()
            
            new_graph = response.json()
            
            return {
                "status": "success",
                "timestamp": datetime.now().isoformat(),
                "new_graph_id": new_graph.get("id"),
                "graph": new_graph,
                "message": f"Successfully added graph: {title}"
            }
    
    except httpx.HTTPStatusError as e:
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "new_graph_id": None,
            "graph": None,
            "message": f"HTTP {e.response.status_code} error: {str(e)}"
        }
    except Exception as e:
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "new_graph_id": None,
            "graph": None,
            "message": f"Error adding graph: {str(e)}"
        }

@tool
async def update_graph(graph_id: str, type: str, sql_query: str, title: Optional[str] = None, extra: Optional[Dict] = None) -> dict:
    """
    Update an existing graph's configuration for better visualization.
    
    Args:
        graph_id (str): ID of graph to update
        type (str): New graph type (bar, line, pie, area, scatter, scatter3d)
        sql_query (str): New SQL query for the graph
        title (Optional[str]): New title for the graph
        extra (Optional[Dict]): Additional parameters like axis labels
    
    Returns:
        dict: Update result with changes made
    """
    try:
        # Prepare update data
        update_data = {
            "type": type,
            "sql_query": sql_query
        }
        
        if title:
            update_data["title"] = title
        
        if extra:
            update_data["extra"] = extra
        
        async with httpx.AsyncClient() as client:
            response = await client.put(
                f"{BASE_URL}/api/graphs/{graph_id}",
                json=update_data,
                timeout=30.0
            )
            response.raise_for_status()
            
            updated_graph = response.json()
            
            return {
                "status": "success",
                "timestamp": datetime.now().isoformat(),
                "updated_graph_id": graph_id,
                "graph": updated_graph,
                "changes": update_data,
                "message": f"Successfully updated graph {graph_id}"
            }
    
    except httpx.HTTPStatusError as e:
        if e.response.status_code == 404:
            return {
                "status": "error",
                "timestamp": datetime.now().isoformat(),
                "updated_graph_id": None,
                "message": f"Graph {graph_id} not found"
            }
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "updated_graph_id": None,
            "message": f"HTTP {e.response.status_code} error: {str(e)}"
        }
    except Exception as e:
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "updated_graph_id": None,
            "message": f"Error updating graph: {str(e)}"
        }

# ============= MODEL & AGENT CONFIGURATION =============

def create_agent(api_key: str):
    """
    Create the financial analysis agent with API tools.
    
    Args:
        api_key (str): Anthropic API key
    
    Returns:
        Agent: Configured agent instance
    """
    model = AnthropicModel(
        client_args={
            "api_key": api_key,
        },
        max_tokens=4096,
        model_id="claude-sonnet-4-20250514",  # Updated to use available model
        params={
            "temperature": 0.3,
        }
    )
    
    # Only include the API-calling tools
    agent = Agent(
        model=model, 
        tools=[
            get_recent_financial_data,
            get_current_graphs,
            remove_graph,
            add_new_graph,
            update_graph
        ]
    )
    
    return agent

# ============= ENHANCED PROMPT WITH EMBEDDED LOGIC =============

def get_enhanced_financial_prompt(max_graphs: int) -> str:
    """
    Returns the comprehensive financial analysis prompt with embedded analysis logic.
    
    Args:
        max_graphs (int): Maximum number of graphs to maintain
        
    Returns:
        str: Detailed financial analysis prompt
    """
    return f"""
You are a sophisticated Financial Data Analysis Agent responsible for maintaining an optimal dashboard of financial insights. Execute the complete graph management workflow with deep analytical rigor:

## GRAPH TYPES & SQL FORMAT REQUIREMENTS

### Available Graph Types:
1. **bar** - Category vs Value comparison
   - SQL Format: (category TEXT, value NUMERIC)
   - Use for: Department expenses, regional comparisons, categorical metrics
   - Extra params: y_axis_label

2. **line** - Time series trends  
   - SQL Format: (time INT8, value NUMERIC)
   - Use for: Revenue over time, customer growth, trend analysis
   - Extra params: x_axis_label, y_axis_label

3. **pie** - Composition/Share of whole
   - SQL Format: (slice TEXT, value NUMERIC)  
   - Use for: Revenue breakdown, market share, portfolio composition

4. **area** - Time series with filled area
   - SQL Format: (time INT8, value NUMERIC)
   - Use for: Cumulative trends, volume over time
   - Extra params: x_axis_label, y_axis_label

5. **scatter** - Correlation/Distribution analysis
   - SQL Format: (x_value NUMERIC, y_value NUMERIC, size NUMERIC NULL, label TEXT NULL)
   - Use for: Risk vs return, cost vs volume correlations
   - Extra params: x_axis_label, y_axis_label

6. **scatter3d** - 3D correlation analysis
   - SQL Format: (x_value NUMERIC, y_value NUMERIC, z_value NUMERIC, size NUMERIC NULL, label TEXT NULL)
   - Use for: Multi-dimensional relationships

## WORKFLOW EXECUTION

### 1. CURRENT GRAPH ANALYSIS & OPTIMIZATION

**Step 1a: Get Current State**
- Use `get_current_graphs()` to retrieve all existing graphs
- Analyze each graph's: id, type, title, sql_query, and current relevance

**Step 1b: Analyze & Update Existing Graphs**
When analyzing graphs, consider:
- **Data Freshness**: Does the query capture the most relevant time periods?
- **Graph Type Optimization**: Is the current chart type optimal for the data pattern?
- **Query Efficiency**: Can the SQL be optimized for better insights?
- **Visual Clarity**: Does the title accurately represent the insight?

If improvements are identified, use `update_graph()` with appropriate parameters.

### 2. COMPREHENSIVE FINANCIAL DATA ANALYSIS

**Step 2a: Data Acquisition**
- Use `get_recent_financial_data(limit=100)` to fetch the latest financial records
- Remember: Maximum 100 rows for analysis (enforced limit)

**Step 2b: Advanced Financial Relationship Discovery**
Analyze the financial data to find interesting relations including:

**FRAUD & RISK ANALYSIS:**
- Transaction amount anomalies (outliers, unusual patterns)
- Frequency spikes in specific categories
- Geographic or temporal risk clustering
- Unusual payment method distributions

**PERFORMANCE METRICS:**
- Revenue trends and growth rates
- Cost efficiency ratios
- Conversion metrics and approval rates
- Profitability by segment/category

**CUSTOMER & MARKET INSIGHTS:**
- Customer segmentation patterns
- Geographic performance variations
- Seasonal or cyclical patterns
- Category/product performance

**CORRELATION ANALYSIS:**
- Multi-variable relationships
- Leading indicators of performance
- Risk factor combinations

### 3. INTELLIGENT GRAPH CREATION

**Step 3a: Relationship Prioritization**
For each discovered relation, evaluate:
- **Business Impact**: How significant for decision-making?
- **Trend Strength**: How clear and consistent is the pattern?
- **Actionability**: Can stakeholders act on this insight?
- **Novelty**: Does this reveal new information not already displayed?

**Step 3b: Optimal Graph Configuration**
Determine the best configuration for high-priority relationships:
- Select appropriate graph type based on data characteristics
- Craft SQL queries that:
  - Return data in the EXACT format required by the graph type
  - Include relevant time filters (e.g., WHERE created_at > NOW() - INTERVAL '30 days')
  - Apply appropriate aggregations (SUM, AVG, COUNT)
  - Include meaningful GROUP BY and ORDER BY clauses
- Generate descriptive, actionable titles

### 4. STRATEGIC GRAPH PORTFOLIO MANAGEMENT

**Step 4a: Portfolio Assessment**
- Count current graphs against maximum limit ({max_graphs})
- If at capacity, identify the least useful graph based on:
  - Data staleness or irrelevance
  - Low information value
  - Redundancy with other graphs
  - Statistical insignificance

**Step 4b: Strategic Removal & Addition**
- If at capacity, use `remove_graph(graph_id)` for the least valuable visualization
- Use `add_new_graph()` with proper parameters:
  - type: One of the available types
  - title: Descriptive, actionable title
  - sql_query: Query returning data in correct format
  - extra: Optional axis labels or other parameters

### 5. COMPREHENSIVE REPORTING

Document your analysis including:
- **Actions Taken**: List all graphs updated, removed, and added
- **Key Insights**: Most significant patterns discovered
- **SQL Queries**: Show the exact queries used for new/updated graphs
- **Graph Configurations**: Detail the `add_new_graph()` parameters used
- **Portfolio Balance**: Ensure coverage across fraud, performance, customer, and market domains

## CRITICAL REQUIREMENTS

1. **SQL Format Compliance**: Each graph type requires specific column names and types in the SQL result
2. **Data Limit**: Never analyze more than 100 rows (use limit=100)
3. **Graph Count**: Maintain exactly {max_graphs} graphs
4. **Tool Usage**: Use the actual tool functions, not mock implementations
5. **Show Configurations**: Always show the exact parameters passed to `add_new_graph()`

Execute this workflow systematically to maintain optimal financial intelligence dashboard.
"""

# ============= MAIN EXECUTION FUNCTIONS =============

def run_graph_management_agent(
    api_key: str,
    interval_seconds: int = 20, 
    max_iterations: Optional[int] = None, 
    max_graphs: int = 6
):
    """
    Runs the enhanced financial graph management agent continuously.
    
    Args:
        api_key (str): Anthropic API key
        interval_seconds (int): How often to run the agent (default: 20 seconds)
        max_iterations (Optional[int]): Maximum number of iterations (None for infinite)
        max_graphs (int): Maximum number of graphs to maintain (default: 6)
    """
    print(f"🚀 Starting Enhanced Financial Data Analysis Agent...")
    print(f"📊 Update interval: {interval_seconds} seconds")
    print(f"📈 Max graphs: {max_graphs}")
    print(f"🔄 Max iterations: {'Unlimited' if max_iterations is None else max_iterations}")
    print(f"🌐 API Base URL: {BASE_URL}")
    print(f"⏰ Start time: {datetime.now()}")
    print("=" * 80)
    
    # Create agent with API key
    agent = create_agent(api_key)
    
    iteration = 0
    
    try:
        while max_iterations is None or iteration < max_iterations:
            iteration += 1
            
            print(f"\n{'='*15} 📊 FINANCIAL ANALYSIS CYCLE {iteration} {'='*15}")
            print(f"🕐 Starting comprehensive analysis at {datetime.now()}")
            
            # Use the enhanced prompt
            message = get_enhanced_financial_prompt(max_graphs)
            
            try:
                result = agent(message)
                print(f"✅ [CYCLE {iteration}] Financial analysis completed successfully")
                
            except Exception as e:
                print(f"❌ [CYCLE {iteration}] Agent error: {e}")
            
            if max_iterations is None or iteration < max_iterations:
                print(f"⏳ [CYCLE {iteration}] Waiting {interval_seconds} seconds until next analysis...")
                print("-" * 80)
                time.sleep(interval_seconds)
    
    except KeyboardInterrupt:
        print(f"\n\n🛑 Received interrupt signal. Stopping after {iteration} iterations.")
        print(f"🏁 End time: {datetime.now()}")
    except Exception as e:
        print(f"\n\n💥 Unexpected error in analysis loop: {e}")
        print(f"🏁 Stopped after {iteration} iterations at {datetime.now()}")

async def run_graph_management_agent_async(
    api_key: str,
    interval_seconds: int = 20, 
    max_iterations: int = 1, 
    max_graphs: int = 6
):
    """
    Async version of the enhanced financial graph management agent.
    
    Args:
        api_key (str): Anthropic API key
        interval_seconds (int): How often to run the agent
        max_iterations (int): Maximum number of iterations  
        max_graphs (int): Maximum number of graphs to maintain
    """
    print(f"🚀 Starting Async Enhanced Financial Analysis Agent...")
    print(f"📊 Update interval: {interval_seconds} seconds")
    print(f"📈 Max graphs: {max_graphs}")
    print(f"🌐 API Base URL: {BASE_URL}")
    print("=" * 80)
    
    # Create agent with API key
    agent = create_agent(api_key)
    
    iteration = 0
    
    try:
        while max_iterations is None or iteration < max_iterations:
            iteration += 1
            
            print(f"\n🔄 [ASYNC CYCLE {iteration}] Starting comprehensive financial analysis...")
            
            # Use the enhanced prompt
            message = get_enhanced_financial_prompt(max_graphs)
            
            try:
                result = await agent.arun(message)
                print(f"✅ [ASYNC CYCLE {iteration}] Financial analysis completed")
                
            except Exception as e:
                print(f"❌ [ASYNC CYCLE {iteration}] Agent error: {e}")
            
            if max_iterations is None or iteration < max_iterations:
                agent.reset()
                await asyncio.sleep(interval_seconds)
    
    except Exception as e:
        print(f"\n💥 Async loop error: {e}")

# ============= MAIN ENTRY POINT =============

if __name__ == "__main__":
    # Get API key from environment or prompt user
    api_key = os.getenv("ANTHROPIC_API_KEY", "")
    
    if not api_key:
        print("⚠️  No ANTHROPIC_API_KEY found in environment variables.")
        api_key = input("Please enter your Anthropic API key: ").strip()
    
    if not api_key:
        print("❌ API key is required to run the agent.")
        exit(1)
    
    print("🎯 Running Enhanced Financial Analysis Agent for 10 iterations...")
    run_graph_management_agent(
        api_key=api_key,
        interval_seconds=20, 
        max_iterations=10, 
        max_graphs=6
    )

   

🎯 Running Enhanced Financial Analysis Agent for 10 iterations...
🚀 Starting Enhanced Financial Data Analysis Agent...
📊 Update interval: 20 seconds
📈 Max graphs: 6
🔄 Max iterations: 10
🌐 API Base URL: http://localhost:8000
⏰ Start time: 2025-09-14 02:22:33.587376

=============== 📊 FINANCIAL ANALYSIS CYCLE 1 ===============
🕐 Starting comprehensive analysis at 2025-09-14 02:22:33.599919
I'll execute the complete graph management workflow with deep analytical rigor. Let me start by analyzing the current state and then proceed systematically through each step.

## Step 1: Current Graph Analysis & Optimization
Tool #1: get_current_graphs
The current dashboard is empty (0 graphs), so we can proceed directly to data analysis and graph creation. This gives us a clean slate to build an optimal 6-graph portfolio.

## Step 2: Comprehensive Financial Data Analysis
Tool #2: get_recent_financial_data
Perfect! I now have 100 financial records to analyze. Let me examine the data structure and identi